<a href="https://colab.research.google.com/github/Shreenidhi-Kovai-Sivabalan/Accurate-Energy-Demand-Prediction/blob/main/InvestigatingMissingValues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Appendix A

# Accurate Energy Demand Prediction for Smart Cities using Deep Learning

Project Code

Shreenidhi Kovai Sivabalan

# A2. Investigating the Missing Values

Further analysis of the missing values to check if they are missing because of a specific reason like a power outage, system failure, sensor issue etc.

In [ ]:
# Mounting to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing the data

filepath = '/content/drive/MyDrive/dissertation/data/household_power_consumption.txt'

data = pd.read_csv(filepath, sep=';', low_memory=False, na_values='?', parse_dates={'Datetime': ['Date', 'Time']}, infer_datetime_format=True)
data.set_index('Datetime', inplace=True)
data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [ ]:
# Printing the number of missing values
# Each column has the exact same number of missing values.
# This warranted further investigation to check if they are Missing Not At Random (MNAR).

print("Missing values in each column: ")
print(data.isnull().sum())

Missing values in each column: 
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


In [ ]:
# Filtering the data points with missing values.

missing_mask = data['Global_active_power'].isna() | \
               data['Global_reactive_power'].isna() | \
               data['Voltage'].isna() | \
               data['Global_intensity'].isna() | \
               data['Sub_metering_1'].isna() | \
               data['Sub_metering_2'].isna() | \
               data['Sub_metering_3'].isna()

In [ ]:
# Creating a dataset containing the missing value data points.

missing_data_df = data[missing_mask].copy()

In [ ]:
# All of the columns have missing values.
# This means that the power consumption was not recorded.
# This could be because of a prolonged power outage or temporary sensor glitch.

missing_data_df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,,,,,
2006-12-21 11:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-21 11:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-30 10:08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-30 10:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-14 18:36:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Printing the first and last 5 rows

print("First 5 rows of missing data:")
print(missing_data_df[['Global_active_power']].head())

print("\nLast 5 rows of missing data:")
print(missing_data_df[['Global_active_power']].tail())

First 5 rows of missing data:
                     Global_active_power
Datetime                                
2006-12-21 11:23:00                  NaN
2006-12-21 11:24:00                  NaN
2006-12-30 10:08:00                  NaN
2006-12-30 10:09:00                  NaN
2007-01-14 18:36:00                  NaN

Last 5 rows of missing data:
                     Global_active_power
Datetime                                
2010-09-28 19:09:00                  NaN
2010-09-28 19:10:00                  NaN
2010-09-28 19:11:00                  NaN
2010-09-28 19:12:00                  NaN
2010-10-24 15:35:00                  NaN


In [ ]:
# Finding the number of unique dates with missing values (no reading).

unique_dates_with_missing_data = pd.Series(missing_data_df.index.date).nunique()
print(f"Number of unique dates with missing data: {unique_dates_with_missing_data}")

Number of unique dates with missing data: 82


The first and last 5 rows were not from one single day. In fact, they were a bit scattered. This points towards temporary glitches.

Further, there are 82 unique days with no reading noted down (mising values). So a single prolonged power outage wasn't the cause.

The scattered nature of the missing data suggests a different underlying cause, such as brief, sporadic sensor disconnections or minor data transmission errors.

In this context, the missing data is more accurately classified as Missing at Random (MAR) or Missing Completely at Random (MCAR). Therefore, interpolation is a reasonable and appropriate method for imputation.